# Рекомендация тарифов

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — мы уже сделали ее, в одном из предыдущих проектов.

Заказчик требует, чтобы у итоговой модели `accuracy` было не меньше 0,75.

## Содержание<a name="Содержание"></a>

1. [Знакомство с данными](#1.-Знакомство-с-данными)
2. [Разбиение данных](#2.-Разбиение-данных)
	1. [Обучающая и тестовая выборки](#Обучающая-и-тестовая-выборки)
	2. [Правило для кросс-валидации](#Правило-для-кросс-валидации)
3. [Исследование моделей](#3.-Исследование-моделей)
	1. [Дерево решений](#Дерево-решений)
	2. [Случайный лес](#Случайный-лес)
	3. [Логистическая регрессия](#Логистическая-регрессия)
	4. [Вывод](#Вывод)
4. [Проверка качества лучшей модели на тестовой выборке](#4.-Проверка-качества-лучшей-модели-на-тестовой-выборке)

## 0. Импорт библиотек

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
%load_ext jupyternotify

<IPython.core.display.Javascript object>

In [3]:
SEED = 42

## 1. Знакомство с данными

Сохраним данные в переменную `df` и посмотрим на них.

In [4]:
df = pd.read_csv('users_behavior.csv')
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
* *сalls* — количество звонков,
* *minutes* — суммарная длительность звонков в минутах,
* *messages* — количество sms-сообщений,
* *mb_used* — израсходованный интернет-трафик в Мб,
* *is_ultra* — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

*is_ultra* — целевой признак.

Изучим общую информацию о датафрейме и проверим, точно ли в нем нет дубликатов.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [6]:
df.duplicated().sum()

0

Видно, что пропусков и дубликатов действительно нет.

[К содержанию](#Содержание)

## 2. Разбиение данных

### Обучающая и тестовая выборки

Сначала поделим датафрейм на признаки и целевой признак. Затем новые датафреймы разобьем на две выборки: обучающую и тестовую. 

Пусть валидационные и тестовая выборка будут равны по размеру и составляют 20% от данных. Тестовую выборку выделим сразу, а валидационные не будем, так как для обучения моделей воспользуемся `GridSearchCV`. В нем реализована кросс-валидация на обучающей выборке.

При разбиении будем стратифицровать выборки по целевому признаку, чтобы во всех выборках была одинаковая пропорция клиентов со «Смартом» и «Ультрой».

In [7]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

In [8]:
train_features, test_features, train_target, test_target = \
    train_test_split(features, target,
                     test_size=0.2,
                     stratify=target,
                     random_state=SEED)

Проверим размеры выборок и баланс классов в них.

In [9]:
def check_split(features, target, n_total):
    class_balance = pd.concat([target.value_counts(),
                               target.value_counts(normalize=True)],
                              axis='columns')
    class_balance.columns = ['absolute', 'share_of_sample']
    
    print(f'Размер выборки: {len(features)}')
    print(f'Доля от всех данных: {len(features)/n_total:.2%}')
    display(class_balance)

In [10]:
check_split(train_features, train_target, len(df))

Размер выборки: 2571
Доля от всех данных: 79.99%


,absolute,share_of_sample
0,1783,0.693504
1,788,0.306496


In [11]:
check_split(test_features, test_target, len(df))

Размер выборки: 643
Доля от всех данных: 20.01%


,absolute,share_of_sample
0,446,0.693624
1,197,0.306376


Данные разделены корректно.

### Правило для кросс-валидации

Мы будем подбирать параметры и обучать модели с помощью `GridSearchCV`. Зададим для него правило по созданию выборок для кросс-валиции.

Ранее мы решили, что валидационные и тестовая выборки будут одинакового размера — 20% от исходных данных. То есть обучающая выборка составляют 80%, и для кросс-валидации мы разобьем ее на четыре части по 20%.

Воспользуемся методом `StratifiedKFold`, чтобы при разбиении данные в блоках были стратифицрованы по целевому признаку.

In [12]:
skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=SEED)

[К содержанию](#Содержание)

## 3. Исследование моделей

У нас есть три алгоритма для обучения модели: дерево решений, случайный лес и логистическая регрессия. Исследуем каждый по порядку.

### Дерево решений

Точность модели, обученной с помощью дерева решений, сильно зависит от высоты дерева. Чтобы понять, какая высота дает лучшую точность, переберем несколько вариантов.

In [13]:
decision_tree = DecisionTreeClassifier(random_state=SEED)

tree_params = {'max_depth': range(1, 51)}

tree_grid_search = GridSearchCV(
    estimator=decision_tree,
    param_grid=tree_params,
    scoring='accuracy',
    cv=skf,
    refit=True)

tree_grid_search.fit(train_features, train_target)

GridSearchCV(cv=StratifiedKFold(n_splits=4, random_state=42, shuffle=True),
             estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_depth': range(1, 51)}, scoring='accuracy')

Выведем высоту лучшей модели и ее `accuracy`. 

In [14]:
def show_best_model_info(model_grid_search, title=''):
    print('\033[1m' + title + ':\033[0m лучший результат и гиперпараметры для него', )
    print('Гиперпараметры:', model_grid_search.best_params_)
    print('Accuracy:', model_grid_search.best_score_)

In [15]:
show_best_model_info(tree_grid_search, 'Дерево решений')

Дерево решений: лучший результат и гиперпараметры для него
Гиперпараметры: {'max_depth': 7}
Accuracy: 0.7965805971812425


### Случайный лес

Для случайного леса будем подбирать не только высоту деревьев, но и их количество.

In [16]:
%%notify

random_forest = RandomForestClassifier(random_state=SEED)

forest_params = {'n_estimators': range(10, 51, 10),
                 'max_depth': range(1, 11)}

forest_grid_search = GridSearchCV(
    estimator=random_forest,
    param_grid=forest_params,
    scoring='accuracy',
    cv=skf,
    refit=True)

forest_grid_search.fit(train_features, train_target)

GridSearchCV(cv=StratifiedKFold(n_splits=4, random_state=42, shuffle=True),
             estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_depth': range(1, 11),
                         'n_estimators': range(10, 51, 10)},
             scoring='accuracy')

<IPython.core.display.Javascript object>

In [17]:
show_best_model_info(forest_grid_search, 'Случайный лес')

Случайный лес: лучший результат и гиперпараметры для него
Гиперпараметры: {'max_depth': 9, 'n_estimators': 20}
Accuracy: 0.8066961236028547


### Логистическая регрессия

Для логистической регрессии сравним Lasso- и Ridge-вариант. То есть попробуем ограничивать сумму модулей и сумма квадратов весов.

In [18]:
logreg = LogisticRegression(solver='liblinear',
                            random_state=SEED)

logreg_params = {'penalty': ['l2', 'l1']}

logreg_grid_search = GridSearchCV(
    estimator=logreg,
    param_grid=logreg_params,
    scoring='accuracy',
    cv=skf,
    refit=True)

logreg_grid_search.fit(train_features, train_target)

GridSearchCV(cv=StratifiedKFold(n_splits=4, random_state=42, shuffle=True),
             estimator=LogisticRegression(random_state=42, solver='liblinear'),
             param_grid={'penalty': ['l2', 'l1']}, scoring='accuracy')

In [19]:
show_best_model_info(logreg_grid_search, 'Логистическая регрессия')

Логистическая регрессия: лучший результат и гиперпараметры для него
Гиперпараметры: {'penalty': 'l1'}
Accuracy: 0.744072881692611


### Вывод

Еще раз посмотрим на результаты обученных моделей:

In [20]:
show_best_model_info(tree_grid_search, 'Дерево решений')

Дерево решений: лучший результат и гиперпараметры для него
Гиперпараметры: {'max_depth': 7}
Accuracy: 0.7965805971812425


In [21]:
show_best_model_info(forest_grid_search, 'Случайный лес')

Случайный лес: лучший результат и гиперпараметры для него
Гиперпараметры: {'max_depth': 9, 'n_estimators': 20}
Accuracy: 0.8066961236028547


In [22]:
show_best_model_info(logreg_grid_search, 'Логистическая регрессия')

Логистическая регрессия: лучший результат и гиперпараметры для него
Гиперпараметры: {'penalty': 'l1'}
Accuracy: 0.744072881692611


Самой точной оказалась модель, обученнная случайным лесом с 20 деревьями высоты 9. Ее `accuracy` больше 0.8.

Второй по точности оказалась модель с одним деревом решений высотой 7. Ее `accuracy` чуть меньше 0.8.

На последнем третьем месте Lasso-вариант логистической регрессией. Она единственная не дотянула до необходимой точности в 0.75.

[К содержанию](#Содержание)

## 4. Проверка качества лучшей модели на тестовой выборке

Посмотрим, какую `accuracy` на тестовой выборке показывает случайный лес.

In [23]:
forest_grid_search.score(test_features, test_target)

0.8258164852255054

Точность на тестовой выборке оказалось даже больше, чем на валидационной. Нас устраивает такой результат.

[К содержанию](#Содержание)